In [5]:
!pip install transformers > /dev/null
!pip install datasets > /dev/null

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric, load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import EarlyStoppingCallback
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()


In [7]:
df = pd.read_csv("data/hard/balanced-reviews.txt", sep="\t", encoding='utf-16')
df.head()

,no,Hotel name,rating,user type,room type,nights,review
0,2,فندق 72,2,مسافر منفرد,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,“ممتاز”. النظافة والطاقم متعاون.
1,3,فندق 72,5,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...
2,16,فندق 72,5,زوج,-,أقمت ليلتين,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...
3,20,فندق 72,1,زوج,غرفة قياسية مزدوجة,أقمت ليلة واحدة,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...
4,23,فندق 72,4,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلتين,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...


In [8]:
df = df[['review', 'rating']]
df

,review,rating
0,“ممتاز”. النظافة والطاقم متعاون.,2
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,5
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,5
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,1
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,4
...,...,...
105693,“فند”. لا شئ عجبني. طقم العمل سيئ جدالا يوجد ب...,1
105694,“سيئ”. قربه من المسجد النبوي الشريف. استخدام م...,2
105695,“اسوأ إقامة في الرحلة !”. القرب من الحرم. كل ش...,2
105696,“دون المستوى”. قربه من الحرم. كل شيء,2


In [10]:
df = df[df.rating != 3]
df

,review,rating
0,“ممتاز”. النظافة والطاقم متعاون.,2
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,5
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,5
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,1
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,4
...,...,...
105693,“فند”. لا شئ عجبني. طقم العمل سيئ جدالا يوجد ب...,1
105694,“سيئ”. قربه من المسجد النبوي الشريف. استخدام م...,2
105695,“اسوأ إقامة في الرحلة !”. القرب من الحرم. كل ش...,2
105696,“دون المستوى”. قربه من الحرم. كل شيء,2


In [11]:
df[df['rating']==3]

,review,rating


In [12]:
df.rename(columns={'review': 'text', 'rating': 'label'}, inplace=True)
df.head()

,text,label
0,“ممتاز”. النظافة والطاقم متعاون.,2
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,5
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,5
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,1
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,4


In [13]:
df['label'].value_counts(normalize=True)

2    0.363933
4    0.250241
5    0.249759
1    0.136067
Name: label, dtype: float64

In [14]:
"""
change class labels positive examples has rating 4,5
negative examples has rating 1,2
rating 3 deleted in above cells
"""
def change_label(label):
    if label==1 or label==2:
        return 0
    else:
        return 1


In [15]:
df['label'] = df['label'].apply(change_label)
df.head()

,text,label
0,“ممتاز”. النظافة والطاقم متعاون.,0
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,1
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,1
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,0
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,1


In [16]:
df['label'].value_counts(normalize=True)

0    0.5
1    0.5
Name: label, dtype: float64

In [17]:
df['label'].unique()

array([0, 1])

In [18]:
X = df['text']
y = df['label']

# create train/test datasets
train_tmp_X, test_X, train_tmp_y, test_y = train_test_split(X, y,   test_size=0.2, random_state=43, stratify=y)
# create train/validation dataset
train_X, val_X, train_y, val_y = train_test_split(train_tmp_X, train_tmp_y, test_size=0.1, random_state=43, stratify=train_tmp_y)

In [19]:
len(train_y), len(val_y), len(test_y)

(76102, 8456, 21140)

In [20]:
#df train
frame_train = { 'text': train_X, 'label': train_y}
df_train = pd.DataFrame(frame_train)
df_train.to_csv("data/hard/train.csv", encoding='utf-8', index=False)

#df val
frame_val = { 'text': val_X, 'label': val_y}
df_val = pd.DataFrame(frame_val)
df_val.to_csv("data/hard/val.csv", encoding='utf-8', index=False)

#df test
frame_test = { 'text': test_X, 'label': test_y }
df_test = pd.DataFrame(frame_test)
df_test.to_csv("data/hard/test.csv", encoding='utf-8', index=False)

In [21]:
df_test.shape

(21140, 2)

In [23]:
df_test['label'].value_counts(normalize=True)

1    0.5
0    0.5
Name: label, dtype: float64

In [24]:
ds = load_dataset('csv', data_files={'train': 'data/hard/train.csv', 'val': 'data/hard/val.csv', 'test': 'data/hard/test.csv'})
ds


Using custom data configuration default-0069275e6995d040


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0069275e6995d040/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 76102
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 8456
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21140
    })
})

In [26]:
model_name = "mofawzy/Bert-hard-balanced"
model_testing = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2)
tokenizer_testing = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/326k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/818k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [27]:
def tokenize(examples):
    outputs = tokenizer_testing(examples['text'], truncation=True,max_length=64)
    return outputs

tokenized_ds = ds.map(tokenize, batched=True)

  0%|          | 0/77 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

In [28]:
test_trainer = Trainer(model=model_testing,tokenizer=tokenizer_testing)
raw_pred, _, _ = test_trainer.predict(tokenized_ds['test'])
y_pred = np.argmax(raw_pred, axis=1)
print(classification_report(test_y, y_pred,digits=4))

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 21140
  Batch size = 8


              precision    recall  f1-score   support

           0     0.9733    0.9547    0.9639     10570
           1     0.9555    0.9738    0.9646     10570

    accuracy                         0.9642     21140
   macro avg     0.9644    0.9642    0.9642     21140
weighted avg     0.9644    0.9642    0.9642     21140

